# Source Restaurant Violations Data

## Imports 

In [1]:
# import pandas, numpy, matplotlib, seaborn 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing the requests library
import requests

import urllib.request
from pathlib import Path
import os

### Resources
 1. [Restaurants in Wake County Data Info](https://www.arcgis.com/home/item.html?id=124c2187da8c41c59bde04fa67eb2872)
 2. [Wake County Open Data](https://data-wake.opendata.arcgis.com/search?tags=restaurants)
 3. [Food Inspection Violations Data Info](https://data.wakegov.com/datasets/Wake::food-inspection-violations/about)
 4. [Wake County Yelp Initiative](https://ash.harvard.edu/news/wake-county-yelp-initiative)

## Fetching restaurant violations from wakegov

In [2]:
## only run this once, this takes a while ##
def getViolationsDf(forceFetch=False):
    path = Path()
    key = 'violations.csv'
    filename = path/key
    
    # If the file does already exist in the directory, download it
    if os.path.exists(filename) and not forceFetch:
        print('Using pre-fetched violations data...')
        df = pd.read_csv(key, dtype={'OBJECTID': 'int64',
         'HSISID': 'int64',
         'INSPECTDATE': 'object',
         'CATEGORY': 'object',
         'STATECODE': 'object',
         'CRITICAL': 'object',
         'QUESTIONNO': 'int64',
         'VIOLATIONCODE': 'object',
         'SEVERITY': 'object',
         'SHORTDESC': 'object',
         'INSPECTEDBY': 'object',
         'COMMENTS': 'object',
         'POINTVALUE': 'float64',
         'OBSERVATIONTYPE': 'object',
         'VIOLATIONTYPE': 'object',
         'CDCDATAITEM': 'object',
         'PERMITID': 'int64'}) # specify dtype to save some memory/time
        print('violations df shape:', df.shape)
        return df
    else:
        print('Fetching restaurant violations data...')
        val = 'https://opendata.arcgis.com/datasets/9b04d0c39abd4e049cbd4656a0a04ba3_2.geojson'
        rows = []
        r = requests.get(url = val)
        data = r.json()
        for d in data:
            data = r.json()['features']
            for d in data:
                rows.append(d['properties'])
        df = pd.DataFrame(rows)
        print('violations df shape:', df.shape)
        df.to_csv(key, index=False)
        print('Done')
        return df

violations_df = getViolationsDf(forceFetch=False)

Using pre-fetched violations data...
violations df shape: (1689108, 18)


# Preprocessing

In [3]:
violations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1689108 entries, 0 to 1689107
Data columns (total 18 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   OBJECTID         1689108 non-null  int64  
 1   HSISID           1689108 non-null  int64  
 2   INSPECTDATE      1689108 non-null  object 
 3   CATEGORY         1689108 non-null  object 
 4   STATECODE        1689108 non-null  object 
 5   CRITICAL         1611268 non-null  object 
 6   QUESTIONNO       1689108 non-null  int64  
 7   VIOLATIONCODE    1689108 non-null  object 
 8   SEVERITY         1611268 non-null  object 
 9   SHORTDESC        1689108 non-null  object 
 10  INSPECTEDBY      1688796 non-null  object 
 11  COMMENTS         1686580 non-null  object 
 12  POINTVALUE       1689108 non-null  float64
 13  OBSERVATIONTYPE  1687840 non-null  object 
 14  VIOLATIONTYPE    899572 non-null   object 
 15  CDCRISKFACTOR    0 non-null        float64
 16  CDCDATAITEM      1

In [4]:
def preprocess_violations(df):
    
    other_cols_to_drop = ['QUESTIONNO', 'COMMENTS', 'COMMENTS', 'CDCDATAITEM', 'VIOLATIONCODE']
    # drop duplicates from dataframe
    df.drop_duplicates(inplace=True)
    
    # drop columns with high missing values
    df = df.drop('CDCRISKFACTOR', axis=1)
    
    # drop irrelevant column(s)
    df = df.drop('STATECODE', axis=1)
    
    # permit id is unique per rest so removing it
    df = df.drop('PERMITID', axis=1)
    
    # normalize observation type
    df['OBSERVATIONTYPE'] = df['OBSERVATIONTYPE'].str.upper()
    df['OBSERVATIONTYPE'].fillna('MISSING', inplace=True)
    
    # convert date to datetime & only get date
    df['INSPECTDATE'] = pd.to_datetime(df['INSPECTDATE']).dt.date
    
    # filter only since beginning of 2017 - we don't want data before that
    df = df[df['INSPECTDATE'] >= pd.to_datetime('2017-01-01').date()]
    
    df.drop(other_cols_to_drop, axis=1, inplace=True)
    
    return df

df = preprocess_violations(violations_df.copy()) # pass in copy for immutability
df.to_csv('preprocessed_violations.csv', index=False)
df.head()

,OBJECTID,HSISID,INSPECTDATE,CATEGORY,CRITICAL,SEVERITY,SHORTDESC,INSPECTEDBY,POINTVALUE,OBSERVATIONTYPE,VIOLATIONTYPE
10,190682112,4092016699,2017-03-24,Approved Source,NaN,NaN,Food obtained from approved source,Angela Myers,0.0,IN,NaN
11,190682241,4092030492,2021-06-14,Approved Source,NaN,NaN,Food obtained from approved source,David Adcock,1.0,OUT,NaN
12,190682242,4092017453,2017-10-18,Approved Source,NaN,NaN,Food obtained from approved source,Maria Powell,0.0,OUT,VR
13,190682243,4092015754,2017-11-15,Approved Source,NaN,NaN,Food obtained from approved source,Maria Powell,0.0,OUT,CDI
20,190682250,4092110015,2018-04-17,Approved Source,NaN,NaN,Food received at proper temperature,Laura McNeill,1.0,OUT,CDI


1.CDI = Corrected during inspection

2.NA = Not Applicable

3.NO = Not Observed

4.R = Repeat Violation

5.VR = Verification Required

## Validation

In [5]:
# critical(handled), severity, inspected by, comments, obs type, violation type, cdcdataitem
display(violations_df.head())
print(violations_df.shape)
print('VIOLATIONTYPE:', violations_df['VIOLATIONTYPE'].unique())
print('OBSERVATIONTYPE:', violations_df['OBSERVATIONTYPE'].unique())
print('CRITICAL:', violations_df['CRITICAL'].unique())
print('CATEGORY:', violations_df['CATEGORY'].unique())
print('SEVERITY', violations_df['SEVERITY'].unique())
print('QUESTIONNO', violations_df['QUESTIONNO'].unique())
print('POINTVALUE', violations_df['POINTVALUE'].unique())
print('CRITICAL COUNTS:', violations_df['CRITICAL'].value_counts(dropna=False))

,OBJECTID,HSISID,INSPECTDATE,CATEGORY,STATECODE,CRITICAL,QUESTIONNO,VIOLATIONCODE,SEVERITY,SHORTDESC,INSPECTEDBY,COMMENTS,POINTVALUE,OBSERVATIONTYPE,VIOLATIONTYPE,CDCRISKFACTOR,CDCDATAITEM,PERMITID
0,190682102,4092015419,2013-01-24T05:00:00Z,Approved Source,".2653,.2655",NaN,9,3-201.11,NaN,Food obtained from approved source,Angela Myers,3-201.11 LG Asafoetida powder is not approved ...,0.0,Out,CDI,NaN,Food shall be obtained from sources that compl...,1490
1,190682103,4092014324,2013-03-04T05:00:00Z,Approved Source,".2653,.2655",NaN,9,3-201.11,NaN,Food obtained from approved source,Melissa Harrison,Pf - 3-201.11 - Several packages of dried good...,0.0,Out,CDI,NaN,Food shall be obtained from sources that compl...,21351
2,190682104,4092040137,2013-03-18T04:00:00Z,Approved Source,".2653,.2655",NaN,9,3-201.11,NaN,Food obtained from approved source,Lisa McCoy,Chicken kabobs are not approved to be on this ...,1.0,Out,NaN,NaN,Food shall be obtained from sources that compl...,22889
3,190682105,4092015740,2013-03-19T04:00:00Z,Approved Source,".2653,.2655",NaN,9,3-201.11,NaN,Food obtained from approved source,Karla Crowder,3-201.11 Provide documentation (receipts) for ...,0.0,In,NaN,NaN,Food shall be obtained from sources that compl...,18960
4,190682106,4092016206,2013-03-27T04:00:00Z,Approved Source,".2653,.2655",NaN,9,3-201.11,NaN,Food obtained from approved source,Melissa Harrison,Pf - 3-201.11 - Habash Shawerma Spices from Ha...,0.0,Out,CDI,NaN,Food shall be obtained from sources that compl...,683


(1689108, 18)
VIOLATIONTYPE: ['CDI' nan 'VR' 'R']
OBSERVATIONTYPE: ['Out' 'In' 'N/O' nan 'OUT' 'IN']
CRITICAL: [nan 'No' 'Yes']
CATEGORY: ['Approved Source' 'Chemical' 'Conformance with Approved Procedures'
 'Consumer Advisory' 'Employee Health'
 'EQUIPMENT AND UTENSILS; PROTECTION FROM CONTAMINATION'
 'Food Identification' 'FOOD PROTECTION' 'Food Temperature Control'
 'Good Hygienic Practices' 'Highly Susceptible Populations'
 'LAUNDRY AND LINENS'
 'MANAGEMENT AND PERSONNEL; EMPLOYEE HEALTH; GOOD HYGIENE PRACTICES; PREVENTING CONTAMINATION BY HANDS'
 'PHYSICAL FACILITIES' 'Physical Facilities'
 'Potentially Hazardous Food Time/Temperature'
 'PREMISES, STORAGE, POISONOUS OR TOXIC MATERIALS'
 'Preventing Contamination by Hands' 'Prevention of Food Contamination'
 'Proper Use of Utensils' 'Protection from Contamination'
 'Safe Food and Water' 'Supervision' 'Utensils and Equipment'
 'WATER, PLUMBING AND WASTE']
SEVERITY [nan 'Core' 'Priority' 'Priority Foundation']
QUESTIONNO [ 9 10 12 11

In [6]:
display(df.isna().sum())
print(df.shape)
display(df.head())

OBJECTID               0
HSISID                 0
INSPECTDATE            0
CATEGORY               0
CRITICAL           10157
SEVERITY           10157
SHORTDESC              0
INSPECTEDBY            0
POINTVALUE             0
OBSERVATIONTYPE        0
VIOLATIONTYPE      99637
dtype: int64

(226604, 11)


,OBJECTID,HSISID,INSPECTDATE,CATEGORY,CRITICAL,SEVERITY,SHORTDESC,INSPECTEDBY,POINTVALUE,OBSERVATIONTYPE,VIOLATIONTYPE
10,190682112,4092016699,2017-03-24,Approved Source,NaN,NaN,Food obtained from approved source,Angela Myers,0.0,IN,NaN
11,190682241,4092030492,2021-06-14,Approved Source,NaN,NaN,Food obtained from approved source,David Adcock,1.0,OUT,NaN
12,190682242,4092017453,2017-10-18,Approved Source,NaN,NaN,Food obtained from approved source,Maria Powell,0.0,OUT,VR
13,190682243,4092015754,2017-11-15,Approved Source,NaN,NaN,Food obtained from approved source,Maria Powell,0.0,OUT,CDI
20,190682250,4092110015,2018-04-17,Approved Source,NaN,NaN,Food received at proper temperature,Laura McNeill,1.0,OUT,CDI


In [7]:
print('OBSERVATIONTYPE:', df['OBSERVATIONTYPE'].unique())
display({column: len(df[column].unique()) for column in df.columns})

OBSERVATIONTYPE: ['IN' 'OUT' 'N/O' 'MISSING']


{'OBJECTID': 226604,
 'HSISID': 5107,
 'INSPECTDATE': 1366,
 'CATEGORY': 25,
 'CRITICAL': 3,
 'SEVERITY': 4,
 'SHORTDESC': 92,
 'INSPECTEDBY': 62,
 'POINTVALUE': 7,
 'OBSERVATIONTYPE': 4,
 'VIOLATIONTYPE': 4}

In [8]:
df.dtypes.apply(lambda x: x.name).to_dict()

{'OBJECTID': 'int64',
 'HSISID': 'int64',
 'INSPECTDATE': 'object',
 'CATEGORY': 'object',
 'CRITICAL': 'object',
 'SEVERITY': 'object',
 'SHORTDESC': 'object',
 'INSPECTEDBY': 'object',
 'POINTVALUE': 'float64',
 'OBSERVATIONTYPE': 'object',
 'VIOLATIONTYPE': 'object'}